# Battle of the Neighborhoods - Capstone Project


## 1. Introduction: Business Problem 

About 300,000 immigrants are coming to Canada every year and a lot of them will settle in Toronto. Many of them need some help to locate the place to live. So, the idea behind this Capstone project is to create a tool which will help people moving to Toronto to choose the right location by providing data about the neighborhoods population, average income, apartment cost, school ranking and venue density. We will be comparing 2 Borrows: North York and Scarborough.

## 2. Data

To solve the problem described above, we need the data on geolocation of neighborhoods, its population, average income in every neighborhood, apartment cost, school ranking and venue density. These data contained in the following data sets:


• __Toronto's Census data__ - is obtained from this website: https://www03.cmhc-schl.gc.ca. It contains the following features:
 - Name of the Neighborhood;
 - Average Household Income Before Taxes;
 - Median Household Income Before Taxes;
 - Average Household Income After Taxes;
 - Median Household Income After Taxes;
    

• __Toronto Neighborhoods data__ - is obtained by scraping Wikipedia page with list of postal_codes of Canada from  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. It contains the following features:
 - Postal Code;
 - Burrows;
 - Neighborhood;
    

• For geotagging Postal Codes in Toronto is used __Geospatial__ dataset with geolocation data, obtained from https://cocl.us/Geospatial_data' by web scraping. It contains the following features:
 - Postal Code;
 - Latitude;
 - Longitude;
    

• __School Ranking Dataset__ - data for both, elementary and secondary schools, obtained from the http://ontario.compareschoolrankings.org/ website through web scraping. It contains the following features:
 - Full name of the school;
 - City;
 - 2017-18 Rating;
 - Rank of the school;
 - Postcode of the school;
    
    
• __Rental Apartments Dataset__ - is obtained from www.kaggle.com and contains the following features:
 - Addresses of apartments;
 - Number of bedrooms;
 - Number of bathrooms;
 - Does it have den?;
 - Postal codes and addresses;
 - Longitude and Latitude;
 - Rental price
    
    
• __Venues Data Dataset__ - is obtained through Foursquare API and contains:
 - Counts of Venues in the Neighborhood;
 - The relative proportion of each venue category;
 - etc.;



When we have all of the data, we can create models, maps and identify clusters according to the following features: population, average income in every neighborhood, apartment cost, school ranking and venue density. 


-------

### 2. Data

#### 2.1. Import libraries

In [2]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge folium=0.5.0 --yes

In [3]:
import folium
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import requests
import re
from geopy.geocoders import Nominatim
import geopy
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

#### 2.1. Building pandas dataframe by scraping Wikipedia page with list of Postal Codes, Burrows and Neighborhoods of Toronto.

In [4]:
response=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

###### Building pandas' dataframe

In [5]:
dfNeighbourhood = pd.read_html(str(response.text))
dfNeighbourhood=dfNeighbourhood[0]

Remove rows with Borough 'Not assigned'

In [6]:
dfNeighbourhood = dfNeighbourhood[dfNeighbourhood['Borough'] != 'Not assigned']
dfNeighbourhood.shape

(210, 3)

Remove duplicates with the same Postcode and list all Boroughs which are having same Postcode, separating them by comma

In [7]:
dfNeighbourhoodGrouped = dfNeighbourhood.groupby(['Postcode','Borough']).agg(', '.join).reset_index()
dfNeighbourhoodGrouped.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
#dfNeighbourhoodGrouped.to_csv('NeighbourhoodGrouped', encoding='utf-8', index=False)

### 2.2 Reading web page with Geospatial data and adding this data to our dataframe with postal codes, neighborhoods and burrows

In [9]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


To merge df and df_geo, use pandas merge method, then drop extra column

In [10]:
dfNeighbourhoodGrouped=pd.merge(dfNeighbourhoodGrouped, df_geo, left_on='Postcode', right_on='Postal Code').drop('Postal Code', axis=1)
dfNeighbourhoodGrouped.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Let's get geographical coordinates of both borough of interest, North York and Scarborough. Then find equal distance point between them. We will use it for visualization purposes.

In [11]:
geopy.geocoders.options.default_user_agent = "my-application"

### North York geolocation

In [12]:
location = 'North York, CA'
geolocator = Nominatim()
location = geolocator.geocode(location)
latitude = location.latitude
longitude = location.longitude
geolocNorthYork = (latitude,longitude)
print('The geolocation of North York, CA is {}.'.format(geolocNorthYork))

The geolocation of North York, CA is (43.7543263, -79.4491169663959).


### Scarborough geolocation

In [13]:
location = 'Scarborough, CA'
geolocator = Nominatim()
location = geolocator.geocode(location)
latitude = location.latitude
longitude = location.longitude
geolocScarborough = (latitude,longitude)
print('The geolocation of Scarborough, CA is {}.'.format(geolocScarborough))

The geolocation of Scarborough, CA is (43.773077, -79.257774).


### Create pandas dataframe with only North York & Scarborough boroughs

In [14]:
#Let's get North York & Scarborough boroughs
northyork_scarborough_neighborhoods = dfNeighbourhoodGrouped[dfNeighbourhoodGrouped.Borough.isin(['North York', 'Scarborough'])].reset_index(drop=True)
northyork_scarborough_neighborhoods.shape

(41, 5)

### Calculate Middle Point location between North York and Scarborough

In [15]:
import operator
geolocMiddlePoint = tuple(map(operator.add, geolocNorthYork, geolocScarborough))
geolocMiddlePoint = tuple(np.array(geolocMiddlePoint)/2.)
#tuple(ti/2 for ti in t)
geolocMiddlePoint

(43.76370165, -79.35344548319796)

### Build the map of both Neighborhoods

In [16]:
map_NorthYork_Scarborough = folium.Map(location=geolocMiddlePoint, zoom_start=11)

# define North York markers
neighborhoods = northyork_scarborough_neighborhoods[northyork_scarborough_neighborhoods.Borough =='North York'].reset_index(drop=True)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], 
                                           neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    
# put North York markers on map
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NorthYork_Scarborough)  
 
# define Scarborough markers
neighborhoods = northyork_scarborough_neighborhoods[northyork_scarborough_neighborhoods.Borough =='Scarborough'].reset_index(drop=True)
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], 
                                           neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    
# put Scarborough markers on map
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='lightred',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NorthYork_Scarborough)  
    
map_NorthYork_Scarborough

#### 2.3 Getting Appartment Rentals dataset from csv file on github

In [17]:
url="https://raw.githubusercontent.com/lorrainedr/Coursera_Capstone/master/Toronto_apartment_rentals_2018.csv"
df_rent = pd.read_csv(url)
df_rent.head()

,Bedroom,Bathroom,Den,Address,Lat,Long,Price
0,2,2.0,0,"3985 Grand Park Drive, 3985 Grand Park Dr, Mis...",43.581639,-79.648193,"$2,450.00"
1,1,1.0,1,"361 Front St W, Toronto, ON M5V 3R5, Canada",43.643051,-79.391643,"$2,150.00"
2,1,1.0,0,"89 McGill Street, Toronto, ON, M5B 0B1",43.660605,-79.378635,"$1,950.00"
3,2,2.0,0,"10 York Street, Toronto, ON, M5J 0E1",43.641087,-79.381405,"$2,900.00"
4,1,1.0,0,"80 St Patrick St, Toronto, ON M5T 2X6, Canada",43.652487,-79.389622,"$1,800.00"


In [18]:
df_rent.shape

(1124, 7)

Extract Postal Code using regular expressions and pandas __apply__ method:

In [19]:
condition = re.compile(r'[A-Z]\d[A-Z] ?\d?[A-Z]?\d?', re.IGNORECASE) # our postal code pattern

def getPostalCode(address): 
    x=condition.search(address)
    if x is None: 
        return np.nan
    else:
        return x.group()[0:3].upper()
          
# Apply above function to Address column, keep extracted PostalCodes in df_rent dataframe 
df_rent['PostalCode'] = df_rent['Address'].apply(getPostalCode)

In [20]:
# For this project we don't use geolocation of appartments and exact addresses, we will use granularity of the PostalCode level
df_rent.drop(['Address', 'Lat', 'Long'], axis=1, inplace=True)

In [21]:
df_rent.shape

(1124, 5)

Before applying __onehotencoder__: lets simplify our dataset by ignoring number of bathrooms and counting den as a bedroom. 
We can do this, because this dataset is used for neighborhood comparison. Of course, if we use this dataset to predict price of renting apartment, we would leave dataset as is, including location.

In [22]:
df_rent["Bedroom"] += df_rent["Den"]
df_rent.drop(['Bathroom', 'Den'], axis=1, inplace=True)
df_rent

,Bedroom,Price,PostalCode
0,2,"$2,450.00",L5B
1,2,"$2,150.00",M5V
2,1,"$1,950.00",M5B
3,2,"$2,900.00",M5J
4,1,"$1,800.00",M5T
...,...,...,...
1119,3,"$3,000.00",L7S
1120,1,"$1,200.00",L6M
1121,1,"$1,800.00",M4C
1122,2,"$2,200.00",M5B


In [23]:
df_rent_onehot = pd.get_dummies(df_rent, columns=['Bedroom'])
df_rent_onehot.Price = [x.strip('$') for x in df_rent_onehot.Price]
df_rent_onehot.Price = df_rent_onehot.Price.apply(lambda x: x.replace(',',''))
df_rent_onehot.Price = pd.to_numeric(df_rent_onehot.Price)
df_rent_onehot.shape

(1124, 5)

Let's rearrange prices by transforming table so bedroom_? column contains price of the apartment:

In [24]:
#df_rent_grouped[['Bedroom_1','Bedroom_2', 'Bedroom_3']] = df_rent_grouped[['Bedroom_1','Bedroom_2', 'Bedroom_3']].apply(pd.to_numeric)
df_rent_onehot.Bedroom_1 = df_rent_onehot.Price.mul(df_rent_onehot.Bedroom_1) 
df_rent_onehot.Bedroom_2 = df_rent_onehot.Price.mul(df_rent_onehot.Bedroom_2)
df_rent_onehot.Bedroom_3 = df_rent_onehot.Price.mul(df_rent_onehot.Bedroom_3)
df_rent_onehot.drop(["Price"], inplace=True, axis=1)

In [25]:
df_rent_onehot.reset_index().drop(["index"], inplace=True, axis=1)
df_rent_onehot

,PostalCode,Bedroom_1,Bedroom_2,Bedroom_3
0,L5B,0.0,2450.0,0.0
1,M5V,0.0,2150.0,0.0
2,M5B,1950.0,0.0,0.0
3,M5J,0.0,2900.0,0.0
4,M5T,1800.0,0.0,0.0
...,...,...,...,...
1119,L7S,0.0,0.0,3000.0
1120,L6M,1200.0,0.0,0.0
1121,M4C,1800.0,0.0,0.0
1122,M5B,0.0,2200.0,0.0


In [26]:
df_rent_onehot.replace(0, np.nan, inplace=True)
df_rent_grouped = df_rent_onehot.groupby(['PostalCode']).agg({'Bedroom_1': ['mean'], 
                                                              'Bedroom_2': ['mean'], 
                                                              'Bedroom_3': ['mean']})
df_rent_grouped.columns = ['Bedroom_1', 'Bedroom_2', 'Bedroom_3']
df_rent_grouped = df_rent_grouped.reset_index()

In [27]:
df_rent_grouped.shape

(115, 4)

In [28]:
df_rent_grouped = pd.merge(dfNeighbourhoodGrouped, df_rent_grouped, how='left', 
                           left_on = 'Postcode', right_on = 'PostalCode')
df_rent_grouped.drop(['PostalCode'], axis=1, inplace=True)

In [29]:
df_rent_grouped = df_rent_grouped.groupby(['Borough']).agg({'Bedroom_1': ['mean'], 
                                                            'Bedroom_2': ['mean'], 
                                                            'Bedroom_3': ['mean']})
df_rent_grouped.columns = ['Bedroom_1', 'Bedroom_2', 'Bedroom_3']
df_rent_grouped = df_rent_grouped.reset_index()

In [30]:
df_rent_grouped = df_rent_grouped.loc[df_rent_grouped['Borough'].isin(['North York', 'Scarborough'])]
df_rent_grouped.reset_index(inplace=True)
df_rent_grouped = df_rent_grouped.drop(['index'], axis=1)
df_rent_grouped

,Borough,Bedroom_1,Bedroom_2,Bedroom_3
0,North York,1439.642857,2083.625,2250.0
1,Scarborough,1200.000000,1898.800,1700.0


#### 2.4 Build Census dataframe from csv file retrieved at https://www03.cmhc-schl.gc.ca

In [31]:
url="https://raw.githubusercontent.com/lorrainedr/Coursera_Capstone/master/CensusModified.csv"
df_census = pd.read_csv(url)
df_census['PostCode'].sort_values(ascending=True)
df_census.shape

(70, 5)

In [32]:
# Let's leave only 'Neighborhood' and 'Median Household Income' columns, 
# dropping both 'after taxes' columns and misleading in this case 'Average Household Income'

df_census.drop(df_census.columns[[0,2,3]], axis=1, inplace=True)
df_census.columns = ["Income", "PostCode"]

In [33]:
df_census

,Income,PostCode
0,"101,482",M1B
1,"56,051",M1C
2,"87,538",M1E
3,"58,921",M1G
4,"65,167",M1H
...,...,...
65,"45,696",M6L
66,"45,211",M6M
67,"73,547",M6N
68,"68,996",M9L


#### 2.5 Build dataframe with Rank and Rating information for elementary schools dataframe. Information was obtained by web scraping http://ontario.compareschoolrankings.org and then iterating home links for every school

In [34]:
res = requests.get("http://ontario.compareschoolrankings.org/elementary/SchoolsByRankLocationName.aspx")
soup = BeautifulSoup(res.content,'lxml')

In [35]:
tds = soup.find_all('td', {'class':'toprow'})
table = tds[0].findParent().findParent()
df_elementary = pd.read_html(str(table),header =0)[0]
#df_elementary.shape

###### We need to retrieve list of links to schools' webpages. For this we use regular expressions to parse table array retrieved above using BeautifulSoup

In [36]:
links = []
baselink = "http://ontario.compareschoolrankings.org"
for link in table.findAll('a', attrs={'href': re.compile("^/elementary/")}):
    links.append(baselink + str(link.get('href')))   

###### Enrich data in main table by visiting each of 3046 schools web page and extracting PostalCode and its full name and then concatenating both tables. Store dataset for future reuse. 

In [37]:
def getSchoolName(span):
    d = re.search(re.escape('<strong>')+"(.*?)"+re.escape('</strong>'), str(span))
    if d:
        return (d.group(1))
    else:
        return (np.nan)

data = pd.DataFrame(links, columns=['url'])
data['fullSchoolName'] = np.nan
data['postalCode'] =np.nan 

for index, row in data.iterrows():
    result = requests.get(row['url'])
    soup = BeautifulSoup(result.content,'lxml')
    span = soup.find("span", id="ctl00_ContentPlaceHolder1_SchoolInfoDisplay")
    schoolAddData.loc[index, ['fullSchoolName', 'postalCode']] = [getSchoolName(str(span)), getPostalCode(str(span))]
    print(data.loc[index])
df_elementary = pd.concat([df_elementary, data], axis=1, join='inner')
df_elementary.to_csv('elementarySchools.csv')

In [38]:
df_elementary = pd.read_csv('elementarySchools.csv',header =0)

In [39]:
df_elementary.shape

(3046, 12)

#### 2.6 Build dataframe with Rank and Rating information for secondary schools dataframe. Information was obtained by web scraping http://ontario.compareschoolrankings.org and then iterating home links for every school

In [40]:
res = requests.get("http://ontario.compareschoolrankings.org/secondary/SchoolsByRankLocationName.aspx")
soup = BeautifulSoup(res.content,'lxml')

In [43]:
tds = soup.find_all('td', {'class':'toprow'})
table = tds[0].findParent().findParent()
df_secondary = pd.read_html(str(table),header =0)[0]
df_secondary

We need to retrieve list of links to schools' webpages. For this we use regular expressions to parse table array retrieved above using BeautifulSoup

In [44]:
links = []
baselink = "http://ontario.compareschoolrankings.org"
for link in table.findAll('a', attrs={'href': re.compile("^/secondary/")}):
    links.append(baselink + str(link.get('href')))

Enrich data in main table by visiting each of 3046 schools web page and extracting PostalCode and its full name and then concatenating both tables. Store dataset for the future reuse. 

In [45]:
import time
data = pd.DataFrame(links, columns=['url'])
data['fullSchoolName'] = np.nan
data['postalCode'] = np.nan 

for index, row in data.iterrows():
    time.sleep(1)
    result = requests.get(row['url'])
    soup = BeautifulSoup(result.content,'lxml')
    span = soup.find("span", id="ctl00_ContentPlaceHolder1_SchoolInfoDisplay")
    data.loc[index, ['fullSchoolName', 'postalCode']] = [getSchoolName(str(span)), getPostalCode(str(span))]
    print(index)
data.shape

In [46]:
df_secondary = pd.concat([df_secondary, data], axis=1, join='inner')
df_secondary.to_csv('secondarySchools.csv')

In [47]:
df_secondary = pd.read_csv('secondarySchools.csv',header =0)
df_secondary.shape

(738, 12)

#### 2.7 Join the data for both types of schools - elementary and secondary. We will keep only data related to location and rating, dropping the rest

In [48]:
dfUnifiedSchools = pd.concat([df_elementary,df_secondary],ignore_index=True, axis=0, sort=False)
cols=[0, 1, 2, 3, 4, 5, 7, 8, 9, 12]
dfUnifiedSchools=dfUnifiedSchools.drop(dfUnifiedSchools.columns[cols], axis=1)

In [49]:
dfUnifiedSchools

,2017-18 Rating,fullSchoolName,postalCode
0,10.0,Al-Risala Academy,L4Y
1,10.0,Avondale Alternative Elementary School,M2N
2,10.0,Havergal College,M5N
3,10.0,IQRA Islamic School,L4Z
4,10.0,Islamic Foundation School Durham,L1Z
...,...,...,...
3779,0.0,Bendale Business &amp; Technical Institute,M1P
3780,0.0,Judith Nyman Secondary School,L6S
3781,0.0,St. Luke Catholic High School,K7A
3782,0.0,Thomas Fiddler Memorial School,P0V


Get average of school rating per location (postal code)

In [50]:
dfUnifiedSchools = dfUnifiedSchools.groupby(['postalCode']).agg({'2017-18 Rating': ['mean']})
dfUnifiedSchools.columns = ['Avg School Rating']
dfUnifiedSchools.reset_index(inplace=True)
dfUnifiedSchools

,postalCode,Avg School Rating
0,H0M,4.900000
1,K0A,6.706452
2,K0B,6.711111
3,K0C,6.432000
4,K0E,5.468750
...,...,...
482,P7K,6.200000
483,P8N,6.075000
484,P8T,2.800000
485,P9A,4.133333


### 2.6 Get North York and Scarborough venues using Foursquare API

In [51]:
northyork_scarborough_neighborhoods.shape

(41, 5)

In [52]:
CLIENT_ID = '0GIRB1FQC1ZZIYJKVTFIF1WC5N01BNXGMAYTKH2TRSGE0UKB' # your Foursquare ID
CLIENT_SECRET = '5WKNPWQS0LPVVIOCDUKZCOMZIEBONO5FW35NMJ4VYLWVBMCU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [53]:
#Let's re-use function we need to explore neighborhoods (see 3-3-2 exercise):
def getNearbyVenues(names, latitudes, longitudes, radius=700, LIMIT = 100):
    print('Getting venues for locations with postal code: ', end='')    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, end=' ')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:

northyork_scarborough_venues = getNearbyVenues(names=northyork_scarborough_neighborhoods['Postcode'],
                                   latitudes=northyork_scarborough_neighborhoods['Latitude'],
                                   longitudes=northyork_scarborough_neighborhoods['Longitude']
                                  )
northyork_scarborough_venues.shape

Getting venues for locations with postal code: M1B M1C M1E M1G M1H M1J M1K M1L M1M M1N M1P M1R M1S M1T M1V M1W M1X M2H M2J M2K M2L M2M M2N M2P M2R M3A M3B M3C M3H M3J M3K M3L M3M M3N M4A M5M M6A M6B M6L M9L M9M 

(562, 7)

### Prepare venues data for the future analysis

In [55]:
northyork_scarborough_venues.head()

,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,M1B,43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
3,M1B,43.806686,-79.194353,Tim Hortons,43.802000,-79.198169,Coffee Shop
4,M1B,43.806686,-79.194353,Lee Valley,43.803161,-79.199681,Hobby Shop


In [56]:
print('There are {} distinct venues in {} categories.'.format(
    len(northyork_scarborough_venues['Venue'].unique()),len(northyork_scarborough_venues['Venue Category'].unique())))

There are 441 distinct venues in 152 categories.


#### Calculate frequencies of venues per postal code

In [57]:
# Apply onehot encoding
venues_onehot = pd.get_dummies(northyork_scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# Add back Postcode and make it first:
venues_onehot['Postcode'] = northyork_scarborough_venues['Postcode']
venues_onehot = venues_onehot[venues_onehot.columns[::-1]]
venues_onehot.columns

Index(['Postcode', 'Women's Store', 'Wings Joint', 'Vietnamese Restaurant',
       'Video Store', 'Video Game Store', 'Trail', 'Toy / Game Store',
       'Thrift / Vintage Store', 'Theater',
       ...
       'Bakery', 'Bagel Shop', 'Badminton Court', 'Auto Garage',
       'Athletics & Sports', 'Asian Restaurant', 'Arts & Crafts Store',
       'American Restaurant', 'Airport', 'Accessories Store'],
      dtype='object', length=153)

In [58]:
# Finally - lets get frequencies per PostalCode
venues_grouped = venues_onehot.groupby('Postcode').mean().reset_index()
venues_grouped.shape

(40, 153)

In [59]:
venues_grouped.tail()

,Postcode,Women's Store,Wings Joint,Vietnamese Restaurant,Video Store,Video Game Store,Trail,Toy / Game Store,Thrift / Vintage Store,Theater,...,Bakery,Bagel Shop,Badminton Court,Auto Garage,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,American Restaurant,Airport,Accessories Store
35,M6A,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.038462,0.0,0.00,0.0,0.0,0.038462
36,M6B,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000
37,M6L,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.166667,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000
38,M9L,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.250000,0.0,0.0,0.0,0.000000,0.0,0.25,0.0,0.0,0.000000
39,M9M,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000


### 2.7 Join all created data sets to have all features in one dataframe

Now we may put together data obtained from open source data and through the Foursquare API. Each dataset has common field - Postal Code.

2.6.1 Average School rating

In [60]:
dfUnified = pd.merge(dfNeighbourhoodGrouped, dfUnifiedSchools, left_on='Postcode', 
                     right_on='postalCode').drop('postalCode', axis=1)
dfUnified.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Avg School Rating
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,6.064000
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,7.090909
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.921053
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5.740000
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,6.571429


2.6.2 Add census data

In [61]:
dfUnified = pd.merge(dfUnified, df_census, left_on='Postcode', 
                     right_on='PostCode').drop('PostCode', axis=1)
dfUnified.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Avg School Rating,Income
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,6.064000,"101,482"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,7.090909,"56,051"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.921053,"87,538"
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5.740000,"58,921"
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,6.571429,"65,167"


2.6.3 Add Venue Data

In [62]:
dfUnified = pd.merge(dfUnified, venues_grouped, left_on='Postcode', 
                     right_on='Postcode')
dfUnified.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Avg School Rating,Income,Women's Store,Wings Joint,Vietnamese Restaurant,...,Bakery,Bagel Shop,Badminton Court,Auto Garage,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,American Restaurant,Airport,Accessories Store
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,6.064000,"101,482",0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,7.090909,"56,051",0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.921053,"87,538",0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5.740000,"58,921",0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,6.571429,"65,167",0.0,0.0,0.0,...,0.111111,0.0,0.0,0.0,0.055556,0.055556,0.0,0.0,0.0,0.0


#### 2.8 Final cleanup/conversion features

In [63]:
dfUnified.Income = dfUnified.Income.apply(lambda x: x.replace(',',''))

In [64]:
dfUnified.iloc[:,5:-1] = dfUnified.iloc[:,5:-1].astype('float64')

## 3. Data Exploration and Analysis

### 3.1 Analyzing neighborhoods by average school rating and average income

Select school rating and average income features (columns 5 and 6)

In [100]:
features = dfUnified.iloc[:,5:7]
features.reset_index(inplace=True, drop=True)

Scale features

In [101]:
sc = StandardScaler().fit(features)
features = sc.transform (features)

Use K-Means algorithm to find the clusters in the dataset based on income and school ratings data. 

In [102]:
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=3).fit(features)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([2, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 2, 0, 1,
       2, 1, 0, 2, 1, 1, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0], dtype=int32)

Combine the cluster labels data with neighborhood data (Postal Code, Latitude, Longitude, Average Income, School Ratings)

In [103]:
dfNonVenue= dfUnified.iloc[:,0:7]
dfNonVenue['Cluster Label'] = kmeans.labels_
dfNonVenue

,Postcode,Borough,Neighborhood,Latitude,Longitude,Avg School Rating,Income,Cluster Label
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,6.064000,101482.0,2
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,7.090909,56051.0,1
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.921053,87538.0,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5.740000,58921.0,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,6.571429,65167.0,1
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,5.180000,49568.0,0
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,5.308333,68524.0,0
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,5.811111,39757.0,0
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,6.281818,75291.0,1
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,6.683333,73216.0,1


### Analyze North York data

In [104]:
# Extract North York for the future processing
dfTempCluster = dfNonVenue[dfNonVenue['Borough']=='North York']
dfTempCluster.reset_index(inplace=True, drop=True)
dfTempCluster.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Avg School Rating,Income,Cluster Label
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,8.200000,74082.0,1
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,5.800000,56644.0,0
2,M2K,North York,Bayview Village,43.786947,-79.385975,4.300000,67355.0,0
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,6.933333,90620.0,2
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,5.300000,61725.0,0


#### Visualize North York clusters

In [105]:
map_clusters = folium.Map(location=geolocNorthYork, zoom_start=11)
# Used Colors
colors=['green', 'blue', 'red']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfTempCluster['Latitude'], dfTempCluster['Longitude'], 
                                  dfTempCluster['Neighborhood'], dfTempCluster['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

---

### Analyze Scarborough data

In [106]:
# Extract North York for the future processing
dfTempCluster = dfNonVenue[dfNonVenue['Borough']=='Scarborough']
dfTempCluster.reset_index(inplace=True, drop=True)
dfTempCluster.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Avg School Rating,Income,Cluster Label
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,6.064000,101482.0,2
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,7.090909,56051.0,1
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.921053,87538.0,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5.740000,58921.0,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,6.571429,65167.0,1


### Vizualize Scarborough cluster data

In [107]:
map_clusters = folium.Map(location=geolocScarborough, zoom_start=11)
# Used Colors
colors=['green', 'blue', 'red', 'black']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfTempCluster['Latitude'], dfTempCluster['Longitude'], 
                                  dfTempCluster['Neighborhood'], dfTempCluster['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

---

### 3.2 Analyzing neighborhoods using Venue data obtain from Foursquare API

Select Venue features (columns from 7 till end)

In [79]:
features = dfUnified.iloc[:,7:-1]
features.reset_index(inplace=True, drop=True)

In [80]:
# Scale features
sc = StandardScaler().fit(features)
features = sc.transform (features)
# print sample of scaled features
features[1] 

array([-0.16222142, -0.17087003, -0.39038939, -0.23075066, -0.16222142,
       -0.16222142, -0.16222142, -0.23099715, -0.23173169, -0.28793627,
       -0.16222142, -0.16222142, -0.30637861, -0.16222142, -0.31071103,
       -0.16222142, -0.20476358, -0.22298539, -0.16222142, -0.16222142,
       -0.16222142, -0.16222142, -0.29127599, -0.31372218, -0.16222142,
       -0.16222142, -0.16222142, -0.31512974, -0.59405224, -0.16222142,
       -0.41561143, -0.33095913, -0.16222142, -0.28057657, -0.16222142,
       -0.16222142, -0.16222142, -0.16222142, -0.16222142, -0.21807671,
       -0.65581887, -0.16222142, -0.49157895, -0.30746991, -0.55558193,
       -0.16222142, -0.16222142, -0.16222142, -0.22973239, -0.16222142,
       -0.23173169, -0.16222142, -0.16222142, -0.16222142, -0.28206837,
       -0.20476358, -0.23201811, -0.16222142, -0.16222142, -0.16222142,
       -0.16222142, -0.16222142, -0.19024748, -0.21053668, -0.16222142,
       -0.22893627, -0.22739034, -0.25969554, -0.36240579, -0.33

Use K-Means algorithm to find the clusters in the dataset with Venues Data

In [87]:
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=3).fit(features)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [88]:
dfVenue= dfUnified.iloc[:,0:5]
dfVenue['Cluster Label'] = kmeans.labels_
dfVenue.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Label
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0


### Analyze North York data

In [89]:
dfTempCluster = dfVenue[dfVenue['Borough']=='North York']
dfTempCluster.reset_index(inplace=True, drop=True)
dfTempCluster.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Label
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,2
2,M2K,North York,Bayview Village,43.786947,-79.385975,0
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,0
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,0


In [90]:
map_clusters = folium.Map(location=geolocNorthYork, zoom_start=11)
# Used Colors
colors=['green', 'blue', 'red']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfTempCluster['Latitude'], dfTempCluster['Longitude'], 
                                  dfTempCluster['Neighborhood'], dfTempCluster['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

---

### Analyze Scarborough data

In [92]:
dfTempCluster = dfVenue[dfVenue['Borough']=='Scarborough']
dfTempCluster.reset_index(inplace=True, drop=True)
dfTempCluster.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Label
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0


In [93]:
map_clusters = folium.Map(location=geolocScarborough, zoom_start=11)
# Used Colors
colors=['green', 'blue', 'red']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfTempCluster['Latitude'], dfTempCluster['Longitude'], 
                                  dfTempCluster['Neighborhood'], dfTempCluster['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

---

## 3.3 Use of Venue data for detailed analysis 

### Top 10 venues categories for both, North York and Scarborough 

In [94]:
#return most common venue categories in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [95]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
northyork_scarborough_venues_sorted = pd.DataFrame(columns=columns)
northyork_scarborough_venues_sorted['Postcode'] = venues_grouped['Postcode']

for ind in np.arange(venues_grouped.shape[0]):
    northyork_scarborough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

northyork_scarborough_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Construction & Landscaping,Spa,Coffee Shop,Hobby Shop,Business Service,Other Great Outdoors,Motorcycle Shop,Movie Theater,Moving Target
1,M1C,Breakfast Spot,Burger Joint,Bar,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target,Noodle House,Other Great Outdoors
2,M1E,Pizza Place,Park,Fast Food Restaurant,Medical Center,Electronics Store,Rental Car Location,Moving Target,Fried Chicken Joint,Bus Line,Mexican Restaurant
3,M1G,Coffee Shop,Park,Business Service,Accessories Store,Other Great Outdoors,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target,Noodle House
4,M1H,Indian Restaurant,Coffee Shop,Bakery,Thai Restaurant,Flower Shop,Fried Chicken Joint,Caribbean Restaurant,Gym / Fitness Center,Chinese Restaurant,Hakka Restaurant


In [96]:
dfTop10Venue = pd.merge(dfVenue, northyork_scarborough_venues_sorted, left_on='Postcode', 
                     right_on='Postcode')
dfTop10Venue

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Construction & Landscaping,Spa,Coffee Shop,Hobby Shop,Business Service,Other Great Outdoors,Motorcycle Shop,Movie Theater,Moving Target
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Breakfast Spot,Burger Joint,Bar,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target,Noodle House,Other Great Outdoors
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Pizza Place,Park,Fast Food Restaurant,Medical Center,Electronics Store,Rental Car Location,Moving Target,Fried Chicken Joint,Bus Line,Mexican Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Park,Business Service,Accessories Store,Other Great Outdoors,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target,Noodle House
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Indian Restaurant,Coffee Shop,Bakery,Thai Restaurant,Flower Shop,Fried Chicken Joint,Caribbean Restaurant,Gym / Fitness Center,Chinese Restaurant,Hakka Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Fast Food Restaurant,Coffee Shop,Convenience Store,Pizza Place,Accessories Store,Outdoor Supply Store,Motorcycle Shop,Movie Theater,Moving Target,Noodle House
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0,Coffee Shop,Discount Store,Chinese Restaurant,Hobby Shop,Bus Line,Department Store,Metro Station,Grocery Store,Sandwich Place,Bus Station
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0,Intersection,Bus Line,Diner,Bakery,Soccer Field,Convenience Store,Metro Station,Fast Food Restaurant,Park,Coffee Shop
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,0,Burger Joint,Wings Joint,Hardware Store,Chinese Restaurant,Outdoor Supply Store,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,Thai Restaurant,Park,General Entertainment,Café,Diner,Skating Rink,College Stadium,Pet Store,Pharmacy,Paper / Office Supplies Store
